In [ ]:
# ===============================================================
# HIGGS BOSON — XGBoost + LightGBM (K-Fold) + AMS + t-SNE (before/after)
# Replaces FTTransformer + SupCon training with gradient-boosted trees.
# ===============================================================

import os, math, zipfile
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import xgboost as xgb
import lightgbm as lgb

# ---------------- Settings ----------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

N_FOLDS = 5
BATCH_SIZE = 512  # not used for trees but kept for parity
EARLY_STOPPING_ROUNDS = 100
XGB_NUM_BOOST_ROUND = 5000
LGB_NUM_BOOST_ROUND = 5000

# Kaggle file handling (paths same as original)
zip_files = {
    "train": "/kaggle/input/higgs-boson/training.zip",
    "test": "/kaggle/input/higgs-boson/test.zip",
    "submission": "/kaggle/input/higgs-boson/random_submission.zip"
}
extract_dir = "/kaggle/working/higgs_data/"
os.makedirs(extract_dir, exist_ok=True)
for key, path in zip_files.items():
    if os.path.exists(path):
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(extract_dir)
        print(f"{key} unzipped.")
    else:
        print(f"{key} zip not found at {path}")

TRAIN_CSV = os.path.join(extract_dir, "training.csv")
TEST_CSV = os.path.join(extract_dir, "test.csv")
OUT_SUB = "/kaggle/working/submission.csv"

# ---------------- AMS Metric ----------------
def ams_score(s, b):
    b_reg = 10.0
    rad = 2.0 * ((s + b + b_reg) * math.log(1.0 + s / (b + b_reg)) - s)
    return math.sqrt(rad) if rad > 0 else 0.0

# ---------------- Load Data & Basic Feature Engineering ----------------
print("Loading data...")
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

train_df.replace(-999.0, np.nan, inplace=True)
test_df.replace(-999.0, np.nan, inplace=True)

# create missing flags as in original
for c in train_df.columns:
    if c in ['EventId', 'Weight', 'Label']:
        continue
    if train_df[c].isna().any():
        train_df[c + '_miss'] = train_df[c].isna().astype(int)
        if c in test_df.columns:
            test_df[c + '_miss'] = test_df[c].isna().astype(int)
        else:
            test_df[c + '_miss'] = 0

numeric_cols = [c for c in train_df.select_dtypes(include=np.number).columns if c != "Weight"]
# fill with median from train
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].median())
num_cols_test = [c for c in numeric_cols if c in test_df.columns]
test_df[num_cols_test] = test_df[num_cols_test].fillna(train_df[num_cols_test].median())

# additional features mirroring original
if {'DER_mass_MMC', 'DER_mass_vis'}.issubset(train_df.columns):
    train_df['mass_ratio'] = train_df['DER_mass_MMC'] / (train_df['DER_mass_vis'] + 1e-6)
    if 'DER_mass_MMC' in test_df.columns and 'DER_mass_vis' in test_df.columns:
        test_df['mass_ratio'] = test_df['DER_mass_MMC'] / (test_df['DER_mass_vis'] + 1e-6)

if {'PRI_tau_pt', 'PRI_met'}.issubset(train_df.columns):
    train_df['pt_ratio'] = train_df['PRI_tau_pt'] / (train_df['PRI_met'] + 1e-6)
    if 'PRI_tau_pt' in test_df.columns and 'PRI_met' in test_df.columns:
        test_df['pt_ratio'] = test_df['PRI_tau_pt'] / (test_df['PRI_met'] + 1e-6)

# labels, weights, event ids
y = (train_df['Label'] == 's').astype(int).values
weights = train_df['Weight'].values
event_ids_test = test_df['EventId'].values

# drop IDs/target columns from features
train_features = train_df.drop(columns=['EventId', 'Weight', 'Label'], errors='ignore')
test_features = test_df.drop(columns=['EventId'], errors='ignore')

# keep consistent columns order
feats = [c for c in train_features.columns if c in test_features.columns or c in train_features.columns]
# make sure same columns appear in test (some engineered could be missing)
test_features = test_features.reindex(columns=train_features.columns, fill_value=0)

# scaling (not necessary for tree models but used for t-SNE stability)
scaler = StandardScaler()
X = scaler.fit_transform(train_features.values.astype(np.float32))
X_test = scaler.transform(test_features.values.astype(np.float32))

# cross-validation
kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

# ---------------- Storage for preds / cv ----------------
oof_xgb = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_ensemble = np.zeros(len(X))
test_pred_xgb = np.zeros(len(X_test))
test_pred_lgb = np.zeros(len(X_test))
test_pred_ensemble = np.zeros(len(X_test))

tsne_embs_before = []
tsne_embs_after = []
tsne_labels = []

# ---------------- Training per fold ----------------
for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
    print(f"\n===== Fold {fold+1}/{N_FOLDS} =====")
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    w_tr, w_va = weights[tr_idx], weights[va_idx]

    # t-SNE "before" on validation raw scaled features (for visualization)
    tsne_embs_before.append(X_va.copy())
    tsne_labels.append(y_va)

    # ---------- XGBoost ----------
    dtrain = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dvalid = xgb.DMatrix(X_va, label=y_va, weight=w_va)
    dtest = xgb.DMatrix(X_test)

    xgb_params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "verbosity": 0,
        "seed": SEED + fold,
        "tree_method": "hist",
        "max_bin": 256,
        "eta": 0.05,
        "max_depth": 6,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "lambda": 1.0,
        "alpha": 0.0,
    }

    xgb_watchlist = [(dtrain, "train"), (dvalid, "valid")]
    xgb_model = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=XGB_NUM_BOOST_ROUND,
        evals=xgb_watchlist,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose_eval=100
    )

    pred_va_xgb = xgb_model.predict(dvalid, iteration_range=(0, xgb_model.best_iteration + 1))
    pred_test_xgb = xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration + 1))

    oof_xgb[va_idx] = pred_va_xgb
    test_pred_xgb += pred_test_xgb / N_FOLDS

    # ---------- LightGBM ----------
    lgb_train = lgb.Dataset(X_tr, label=y_tr, weight=w_tr)
    lgb_valid = lgb.Dataset(X_va, label=y_va, weight=w_va, reference=lgb_train)

    lgb_params = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "seed": SEED + fold,
        "boosting_type": "gbdt",
        "learning_rate": 0.05,
        "num_leaves": 63,
        "max_depth": 10,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "lambda_l1": 0.0,
        "lambda_l2": 1.0,
    }

    lgb_model = lgb.train(
        params=lgb_params,
        train_set=lgb_train,
        num_boost_round=LGB_NUM_BOOST_ROUND,
        valid_sets=[lgb_train, lgb_valid],
        valid_names=["train", "valid"],
        callbacks=[
            lgb.early_stopping(EARLY_STOPPING_ROUNDS, verbose=True),
            lgb.log_evaluation(100)
        ]
    )


    pred_va_lgb = lgb_model.predict(X_va, num_iteration=lgb_model.best_iteration)
    pred_test_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

    oof_lgb[va_idx] = pred_va_lgb
    test_pred_lgb += pred_test_lgb / N_FOLDS

    # ---------- Ensemble (simple average) ----------
    pred_va_ens = 0.5 * pred_va_xgb + 0.5 * pred_va_lgb
    pred_test_ens = 0.5 * pred_test_xgb + 0.5 * pred_test_lgb  # note: test_pred_xgb/lgb already being averaged across folds
    oof_ensemble[va_idx] = pred_va_ens
    # we accumulate test_pred_ensemble once per fold after we update test_pred_xgb/lgb at the top of loop
    # To keep consistency, compute test fold contributions directly from this fold's predictions:
    test_pred_ensemble += 0.5 * pred_test_xgb + 0.5 * pred_test_lgb

    # ---------- diagnostics ----------
    va_auc_xgb = roc_auc_score(y_va, pred_va_xgb)
    va_auc_lgb = roc_auc_score(y_va, pred_va_lgb)
    va_auc_ens = roc_auc_score(y_va, pred_va_ens)
    print(f"Fold {fold+1} AUCs -> XGB: {va_auc_xgb:.5f}, LGB: {va_auc_lgb:.5f}, ENS: {va_auc_ens:.5f}")

    # t-SNE "after" - augment validation features with ensemble probability as an extra dimension
    X_va_after = np.hstack([X_va, pred_va_ens.reshape(-1, 1)])
    tsne_embs_after.append(X_va_after)

# finalize test ensemble averaging
# test_pred_xgb and test_pred_lgb are already averaged across folds; test_pred_ensemble was accumulated fold-wise (we added per-fold predictions)
# Normalize test_pred_ensemble (we added N_FOLDS times fold-specific half-averages):
test_pred_ensemble = test_pred_ensemble / N_FOLDS

# ---------------- t-SNE Visualization ----------------
# Concatenate per-fold arrays for visualization (before: raw features; after: features + pred)
X_before_all = np.vstack(tsne_embs_before)
X_after_all = np.vstack(tsne_embs_after)
y_tsne = np.concatenate(tsne_labels)

print("Running t-SNE on 'before' (raw scaled validation features)...")
tsne = TSNE(n_components=2, perplexity=50, random_state=SEED, init='pca', n_iter=1000)
X2d_before = tsne.fit_transform(X_before_all)
plt.figure(figsize=(7,6))
plt.scatter(X2d_before[:,0], X2d_before[:,1], c=y_tsne, s=6, alpha=0.7)
plt.title("t-SNE of Validation Features — BEFORE")
plt.xlabel("TSNE-1"); plt.ylabel("TSNE-2")
plt.show()

print("Running t-SNE on 'after' (validation features + ensemble prob)...")
tsne2 = TSNE(n_components=2, perplexity=50, random_state=SEED, init='pca', n_iter=1000)
X2d_after = tsne2.fit_transform(X_after_all)
plt.figure(figsize=(7,6))
plt.scatter(X2d_after[:,0], X2d_after[:,1], c=y_tsne, s=6, alpha=0.7)
plt.title("t-SNE of Validation Features + Ensemble Prob — AFTER")
plt.xlabel("TSNE-1"); plt.ylabel("TSNE-2")
plt.show()

# ---------------- OOF Metrics & AMS Optimization ----------------
# Use the ensemble oof predictions for thresholding / AMS
oof_preds = oof_ensemble.copy()
overall_auc = roc_auc_score(y, oof_preds)
print(f"\nOverall OOF AUC (ensemble) = {overall_auc:.5f}")

thr_range = np.linspace(0.01, 0.99, 99)
best_thr, best_ams = 0.5, -1
for t in thr_range:
    s = weights[(y == 1) & (oof_preds > t)].sum()
    b = weights[(y == 0) & (oof_preds > t)].sum()
    sc = ams_score(s, b)
    if sc > best_ams:
        best_ams, best_thr = sc, t
print(f"Best AMS on OOF = {best_ams:.3f} @ thr={best_thr:.4f}")

# ---------------- Prepare Submission ----------------
print("\nWriting submission...")
# use the ensemble test predictions (we built test_pred_ensemble)
# Ensure test_pred_ensemble length matches test set
if len(test_pred_ensemble) != len(event_ids_test):
    # as a fallback, average the per-model averaged predictions:
    test_pred_ensemble = 0.5 * test_pred_xgb + 0.5 * test_pred_lgb

rankorder = np.argsort(np.argsort(test_pred_ensemble)) + 1
classes = np.where(test_pred_ensemble > best_thr, 's', 'b')
sub = pd.DataFrame({"EventId": event_ids_test, "RankOrder": rankorder, "Class": classes})
sub.to_csv(OUT_SUB, index=False)
print("Saved submission to:", OUT_SUB)
print("Final AMS:", best_ams)
